In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('RR_company.csv',encoding='latin-1')
cn = list(data[data['headquarter_country']=='China']['name'])

Some names are accompanied with their former names, which may result in unsuccessful webscraping, so all content within the parentheses are removed.

In [ ]:
for i in range(len(cn)):
  if '(' in cn[i]:
    idx_beg = cn[i].find('(')
    cn[i] = cn[i][:idx_beg]

print(cn)

### Start scraping

In [ ]:
### uncomment the commands below if selenium is not installed
# !pip install selenium
# !apt-get update 
# !apt install chromium-chromedriver

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
driver =webdriver.Chrome('chromedriver',chrome_options=chrome_options)

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

     |████████████████████████████████| 911kB 7.6MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:12 https://developer.download.nvidia.com/compute/cuda/repos/ubu

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
options = Options()
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")

Since this is implemented on Google Colab and pretty time-consuming, it is safer to break the list of names into chunks and store them to avoid inconvenience caused by runtime limit and other issues.

In [ ]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
name_chunks = list(chunks(cn,100))

In [ ]:
cnt = 10
for chunk in name_chunks[9:]:
  dict_code = {}
  for name in chunk:
    driver =webdriver.Chrome('chromedriver',options=chrome_options)
    driver.get('https://finance.yahoo.com/')

    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//input[@name='yfin-usr-qry']"))).send_keys(name)
    try:
      yahoo_fin_auto_suggestions = WebDriverWait(driver, 20).until(EC.visibility_of_all_elements_located((By.XPATH, '//*[@id="header-search-form"]/div[2]/div[1]/div/ul[2]/li')))
      code = yahoo_fin_auto_suggestions[0].text[:9]
      dict_code[name] = code
      print(code)
    except:
      dict_code[name] = 'Code Not Found'
    driver.quit()
  df = pd.DataFrame(dict_code, index=[0]).T
  df.columns = ['code']
  df.to_excel('chunk_{}.xlsx'.format(cnt))  
  cnt += 1


### Clean scraped data

In [ ]:
df = pd.read_excel('chunk_1.xlsx')
df.columns = ['company_name','code']
for i in range(2,17):
  tmp = pd.read_excel('chunk_{}.xlsx'.format(i))
  tmp.columns = ['company_name','code']
  df = df.append(tmp, ignore_index=True)
df

,company_name,code
0,Kangmei Pharmaceutical Co Ltd,600518.SS
1,Jiangsu Xiuqiang Glasswork Co Ltd,300160.SZ
2,Henan Dayou Energy Co Ltd,600403.SS
3,Joyvio Agriculture Development Co Ltd,300268.SZ
4,AAC Technologies Holdings Inc,Edited Tr
...,...,...
1560,Beijing Honggao Creative Architectural Design ...,002504.SZ
1561,Jiangling Motors Corp Ltd,Code Not Found
1562,Leshi Internet Information & Technology Corp,Code Not Found
1563,AutoNavi Holdings Ltd,Code Not Found


### Scrape from Google Finance

In [ ]:
import time # WebDriverWait is actually a much better alternative but this does the job

In [ ]:
for i in range(1,17):
  tmp_df = pd.read_excel('chunk_{}.xlsx'.format(i))
  tmp_df.columns = ['company_name','code']
  dict_code = {}
  for j in range(len(tmp_df)):
    code = tmp_df['code'].iloc[j]
    company = tmp_df['company_name'].iloc[j]

    if code[-2:] == 'SZ' or code[-2:] == 'SS':
       dict_code[company] = code
    else:
      try:
        driver = webdriver.Chrome('chromedriver',options=chrome_options)
        driver.get('https://google.com/finance')  
        search = driver.find_element_by_xpath('//*[@id="search-bar"]')
        search.send_keys(company)
        search.send_keys(Keys.RETURN)
        time.sleep(1.5) 
        new_code = driver.current_url[37:]
        dict_code[company] = new_code
        print(new_code)
      except:
        dict_code[company] = 'Not Found'
    driver.quit()
  df = pd.DataFrame(dict_code, index=[0]).T
  df.columns = ['code']
  df.to_excel('new_chunk_{}.xlsx'.format(i))  
  
  

In [ ]:
df = pd.read_excel('new_chunk_1.xlsx')
for i in range(2,17):
  tmp = pd.read_excel('new_chunk_{}.xlsx'.format(i))
  df = df.append(tmp, ignore_index=True)

In [ ]:
df.to_excel('full_rr_company.xlsx')

Companies that still have no code (cannot find on Google or Yahoo Finance) might need more effort, so I attempted to scrape from MarketScreener but there are still about 200 companies that cannot be matched with a code, and I decided that this is good enough for now.<br>
Below is the code used for scraping from MarketScreener.

In [ ]:
scraped_comp = pd.read_excel('full_rr_company.xlsx')
# scraped_comp = scraped_comp[['company','code']].set_index('company')

In [ ]:
# dict_code = scraped_comp.to_dict()['code']
# dict_code

In [ ]:
# cnt=0
# for comp, code in dict_code.items():
#   if type(code)!=str :
#     cnt+=1
# print('There are still {} companies with no code'.format(cnt))
# scrape.head()

There are still 270 companies with no code


In [ ]:
# cnt = 0
# for comp, code in dict_code.items():
#   cnt += 1
#   print(cnt)
#   if type(code)!=str:
#     try:
#       print(comp)
#       driver = webdriver.Chrome('chromedriver',options=chrome_options)
#       driver.get('https://www.marketscreener.com/')  
#       search = driver.find_element_by_xpath('//*[@id="autocomplete"]')
#       search.send_keys(comp)
#       search.send_keys(Keys.RETURN)
#       result = WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="ALNI0"]/tbody/tr[2]/td[1]')))
#       new_code = result.text
#       exc = driver.find_element_by_xpath('//*[@id="ALNI0"]/tbody/tr[2]/td[7]').text
#       if exc.lower()[:5] == 'shang':
#         new_code = new_code+'.SH'
#       elif exc.lower()[:5] == 'shenz':
#         new_code = new_code+'.SZ'
#       else:
#         new_code = new_code +'.'+ exc
#       dict_code[comp] = new_code
#       print(new_code)
#     except:
#       continue

### Add the scraped code as a new column to the original RepRisk data

In [ ]:
rr_company_cn = data[data['headquarter_country']=='China']
rr_company_cn['code'] = list(scraped_comp['code'])
rr_company_cn.to_csv('RepRisk_Chinese_Companies_info.csv')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### Get intersection

In [ ]:
code = scraped_comp['code']

In [ ]:
syntao = pd.read_excel('SynTao_clean.xlsx')
st_code = list(syntao.code)
def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 
print('There are {} companies that both RepRisk and Syntao include that can be used for further comparison'.format(len(intersection(code,st_code))))

There are 384 companies that both RepRisk and Syntao include that can be used for further comparison
